In [1]:
%load_ext autoreload
%autoreload 2


from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    

load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/w{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 25), 
        datetime.date(2020, 1, 20),
        datetime.timedelta(weeks=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/w{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 1, 20), 
        datetime.date(2020, 2, 3),
        datetime.timedelta(weeks=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((283833, 36), (283833, 4), (75767, 36), (75767, 4))

In [6]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((36, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 34, 64)            256       
                                                                 
 flatten_1 (Flatten)         (None, 2176)              0         
                                                                 
 dense_2 (Dense)             (None, 8)                 17416     
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17,681
Trainable params: 17,681
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/w-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_view])

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/w-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_cart])

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/w-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_remove_from_cart])

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/w-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_purchase])

Epoch 1/10
8870/8870 [==============================] - 17s 2ms/step - loss: 753.0549 - root_mean_squared_error: 27.4418 - val_loss: 1377.7699 - val_root_mean_squared_error: 37.1183
Epoch 2/10
8870/8870 [==============================] - 18s 2ms/step - loss: 334.2260 - root_mean_squared_error: 18.2818 - val_loss: 1409.8708 - val_root_mean_squared_error: 37.5482
Epoch 3/10
8870/8870 [==============================] - 20s 2ms/step - loss: 334.1876 - root_mean_squared_error: 18.2808 - val_loss: 1449.3274 - val_root_mean_squared_error: 38.0700
Epoch 4/10
8870/8870 [==============================] - 20s 2ms/step - loss: 332.8439 - root_mean_squared_error: 18.2440 - val_loss: 1138.5463 - val_root_mean_squared_error: 33.7424
Epoch 5/10
8870/8870 [==============================] - 23s 3ms/step - loss: 295.5320 - root_mean_squared_error: 17.1910 - val_loss: 1797.5740 - val_root_mean_squared_error: 42.3978
Epoch 6/10
8870/8870 [==============================] - 20s 2ms/step - loss: 314.0808 - ro

In [8]:
model_view.save('../model/saved/w-view.keras')
model_cart.save('../model/saved/w-cart.keras')
model_remove_from_cart.save('../model/saved/w-remove_from_cart.keras')
model_purchase.save('../model/saved/w-purchase.keras')